# **Import Some Packages**

In [436]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For data preprocess
import pandas as pd
import numpy as np
import csv
import os

from tqdm import tqdm

import math
from torch.utils.tensorboard import SummaryWriter
# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
    
from sklearn.preprocessing import StandardScaler

# **Some Utilities**

You do not need to modify this part.

In [2]:
def get_device():
    ''' Get device (if GPU is available, use GPU) '''
    return 'cuda' if torch.cuda.is_available() else 'cpu'

def plot_learning_curve(loss_record, title=''):
    ''' Plot learning curve of your DNN (train & dev loss) '''
    total_steps = len(loss_record['mean_train_loss'])
    x_1 = range(total_steps)
    x_2 = x_1[::len(loss_record['mean_train_loss']) // len(loss_record['mean_valid_loss'])]
    figure(figsize=(6, 4))
    plt.plot(x_1, loss_record['mean_train_loss'], c='tab:red', label='train')
    plt.plot(x_2, loss_record['mean_valid_loss'], c='tab:cyan', label='dev')
    plt.ylim(0,3)
    plt.xlabel('Training steps')
    plt.ylabel('MSE loss')
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.show()


def plot_pred(dv_set, model, device, lim=35., preds=None, targets=None):
    ''' Plot prediction of your DNN '''
    if preds is None or targets is None:
        model.eval()
        preds, targets = [], []
        for x, y in dv_set:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                preds.append(pred.detach().cpu())
                targets.append(y.detach().cpu())
        preds = torch.cat(preds, dim=0).numpy()
        targets = torch.cat(targets, dim=0).numpy()

    figure(figsize=(5, 5))
    plt.scatter(targets, preds, c='r', alpha=0.5)
    plt.plot([-0.2, lim], [-0.2, lim], c='b')
    plt.xlim(-0.2, lim)
    plt.ylim(-0.2, lim)
    plt.xlabel('ground truth value')
    plt.ylabel('predicted value')
    plt.title('Ground Truth v.s. Prediction')
    plt.show()
    


In [438]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

In [439]:
attri_data = pd.read_csv('../LiterallyWikidata/files_needed/numeric_literals_ver06')

In [440]:
attri_data

,e,a,v,name_e,name_a,ent_type,std_v,minmax_v
0,Q1000056,P1082,1.103200e+04,Sušice,population,Q7841907,-0.030708,2.649215e-06
1,Q1000056,P2044,4.720000e+02,Sušice,elevation above sea level,Q7841907,0.031306,2.219876e-03
2,Q1000056,P2046,4.563000e+07,Sušice,area,Q7841907,-0.006852,7.492611e-12
3,Q1000138,P1082,1.375000e+03,Cantenac,population,Q484170,-0.031014,3.301914e-07
4,Q1000138,P2044,1.000000e+00,Cantenac,elevation above sea level,Q484170,-0.116549,1.058141e-03
...,...,...,...,...,...,...,...,...
296303,Q99987,P1333_Longtiude,9.586108e+00,Brembate di Sopra,coordinates of southernmost point,Q747074,-0.140675,5.334437e-01
296304,Q99987,P1334_Longtiude,9.595788e+00,Brembate di Sopra,coordinates of easternmost point,Q747074,-0.144653,5.264729e-01
296305,Q99987,P1335_Longtiude,9.564391e+00,Brembate di Sopra,coordinates of westernmost point,Q747074,-0.142728,5.252904e-01
296306,Q99987,P625_Longtiude,9.580647e+00,Brembate di Sopra,coordinate location(logtitude),Q747074,0.130122,5.244460e-01


In [441]:
# 用kgeemb順序
ent2idx ={}
with open('../LiterallyWikidata/files_needed/list_ent_ids.txt','r') as fr:
    for i, word in enumerate(fr.readlines()):
        ent2idx[word.strip()] = i


attri_data_std_v = attri_data[['e','a','std_v']]
attri_data_eav = attri_data[['e','a','v']]
# att2idx = {}
# #rel2idx = {v:k for k,v in enumerate(relations['label'].unique())}

# with open('../LiterallyWikidata/files_needed/attribute.txt','r') as fr:
#     for i, word in enumerate(fr.readlines()):
#         att2idx[word.strip()] = i
        
att2idx = {v:k for k,v in enumerate(attri_data['a'].unique())}

In [442]:
# attri_data['a_idx']=attri_data['a'].map(att2idx)
# attri_data['e_idx']=attri_data['e'].map(ent2idx)

In [443]:
attri_data_eav

,e,a,v
0,Q1000056,P1082,1.103200e+04
1,Q1000056,P2044,4.720000e+02
2,Q1000056,P2046,4.563000e+07
3,Q1000138,P1082,1.375000e+03
4,Q1000138,P2044,1.000000e+00
...,...,...,...
296303,Q99987,P1333_Longtiude,9.586108e+00
296304,Q99987,P1334_Longtiude,9.595788e+00
296305,Q99987,P1335_Longtiude,9.564391e+00
296306,Q99987,P625_Longtiude,9.580647e+00


In [444]:
def numeric_literal_array(data, ent2idx, att2idx):
    #'LiterallyWikidata/LitWD48K/train_attri_data'
    df_all = data

    # Resulting file
    num_lit = np.zeros([len(ent2idx), len(att2idx)], dtype=np.float32)

# Create literal wrt vocab
    for i, (s, p, lit) in enumerate(df_all.values):
        try:
            num_lit[ent2idx[s], att2idx[p]] = lit
        except KeyError:
            continue
    return num_lit


# num_lit shape (47998, 86)


In [445]:
num_lit = numeric_literal_array(attri_data_eav, ent2idx, att2idx)
print(num_lit.shape)

(47998, 86)


In [446]:
num_lit_stdv = numeric_literal_array(attri_data_std_v, ent2idx, att2idx)
print(num_lit_stdv.shape)

(47998, 86)


In [447]:
# ## constraint needed:
pop_idx = att2idx['P1082']
gdp = att2idx['P4010']
nominal_gdp = att2idx['P2131']
# nominal_gdp_per = att2idx['P2132']
gdp_per = att2idx['P2299']
# date_of_birth = att2idx['P569']
# date_of_death = att2idx['P570']
# area = ['P2046']
# # net_profit = att2idx['P2295']
# # retirement_age = att2idx['P3001']
# # age_of_majority = att2idx['P2997']
# # work_start = att2idx['P2031']
# # work_end = att2idx['P2032']

In [448]:
ent2idx['Q1000']

84

In [449]:
attri_data[attri_data['a']=='P1082']

,e,a,v,name_e,name_a,ent_type,std_v,minmax_v
0,Q1000056,P1082,11032.0,Sušice,population,Q7841907,-0.030708,2.649215e-06
3,Q1000138,P1082,1375.0,Cantenac,population,Q484170,-0.031014,3.301914e-07
6,Q100013,P1082,4109.0,Brembilla,population,Q1134686,-0.030927,9.867318e-07
9,Q100015,P1082,6009.0,Brignano Gera d'Adda,population,Q747074,-0.030867,1.442996e-06
12,Q100016,P1082,119.0,Brumano,population,Q747074,-0.031053,2.857656e-08
...,...,...,...,...,...,...,...,...
123807,Q99983,P1082,706.0,Bracca,population,Q747074,-0.031035,1.695383e-07
123810,Q99985,P1082,712.0,Branzi,population,Q747074,-0.031035,1.709791e-07
123813,Q99986,P1082,8551.0,Brembate,population,Q747074,-0.030787,2.053430e-06
123816,Q99987,P1082,7868.0,Brembate di Sopra,population,Q747074,-0.030808,1.889415e-06


In [450]:
num_lit[84][gdp_per]*num_lit[84][pop_idx]
num_lit[84][gdp]

36681910000.0

In [451]:
# x_list: ent的gdp有值，把除了gdp那個值之外的值存到inner_x_list，len(x_list)是有幾組變數有值
# normalized or non-normalized
x_list=[]

for ent in num_lit:
    if ent[gdp] == 0:
        pass
    else:
        #print(ent[gdp])
        inner_x_list=[]
        for j in range(len(ent)):
            if j != gdp :
                inner_x_list.append(ent[j])
        inner_x_list.append(ent[gdp])
        x_list.append(inner_x_list)


In [452]:
select_feature = []
for i in range(len(x_list[0])-1):
    if x_list[0][i] !=0:
        select_feature.append(i)

In [453]:
pd.DataFrame(x_list,columns=list(range(len(x_list[0]))))

,0,1,2,3,4,5,6,7,8,9,...,76,77,78,79,80,81,82,83,84,85
0,325145952.0,0.0,9.826675e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,71.379997,25.837132,44.815392,65.637886,-98.579498,-156.479996,-97.394829,-66.949951,-168.118286,1.948539e+13
1,83149296.0,0.0,3.574000e+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,54.911209,47.270027,51.272800,51.051102,10.000000,8.669200,10.178447,15.041781,5.866366,4.345631e+12
2,2025137.0,0.0,2.676670e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.320000,-3.960050,-0.617460,-0.624444,11.500000,11.700000,11.153220,14.526600,8.708055,3.668191e+10
3,19586540.0,0.0,2.383970e+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.270000,43.623371,45.163815,46.121685,25.000000,26.700001,25.391186,29.716679,20.261787,5.221131e+11
4,42558328.0,0.0,6.036290e+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,52.379528,44.386417,49.260277,48.419167,32.000000,33.190605,33.777222,40.228333,22.137222,3.695664e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,807610.0,0.0,3.839400e+10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28.320000,26.702021,27.281780,27.141870,90.500000,90.000000,89.770851,92.125229,88.746468,7.584701e+09
152,777859.0,0.0,2.149700e+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.530000,1.164920,1.958640,5.962424,-59.316666,-59.980000,-58.825779,-56.491150,-61.406292,6.362827e+09
153,341465152.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.539088e+13
154,5260750.0,0.0,3.420000e+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.700000,0.000000,0.000000,0.000000,15.383330,17.480000,0.000000,0.000000,0.000000,2.869352e+10


In [454]:
y_list=list()
for ent2 in num_lit:
    if ent2[gdp] ==0:
        pass
    else:
        y_list.append(ent2[gdp])

In [455]:
for i in range(len(x_list)):
    inner_x = x_list[i]
    inner_x.append(y_list[i])
x_list.append(inner_x)


In [456]:
select_feature

[0,
 2,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 65,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84]

In [457]:
attri_data[attri_data['a']=='P4010']

,e,a,v,name_e,name_a,ent_type,std_v,minmax_v
156,Q1000,P4010,3.668191e+10,Gabon,GDP (PPP),Q3624078,-0.225985,0.001880
414,Q1005,P4010,3.569106e+09,The Gambia,GDP (PPP),Q6256,-0.241117,0.000181
461,Q1006,P4010,2.857575e+10,Guinea,GDP (PPP),Q6256,-0.229690,0.001464
519,Q1007,P4010,3.171312e+09,Guinea-Bissau,GDP (PPP),Q6256,-0.241299,0.000160
588,Q1008,P4010,9.583674e+10,Ivory Coast,GDP (PPP),Q7270,-0.198953,0.004916
...,...,...,...,...,...,...,...,...
121853,Q971,P4010,2.869352e+10,Republic of the Congo,GDP (PPP),Q6256,-0.229636,0.001470
122011,Q974,P4010,7.231902e+10,Democratic Republic of the Congo,GDP (PPP),Q6256,-0.209700,0.003709
122243,Q977,P4010,2.342711e+09,Djibouti,GDP (PPP),Q6256,-0.241678,0.000118
122774,Q983,P4010,3.098132e+10,Equatorial Guinea,GDP (PPP),Q3624078,-0.228591,0.001588


In [458]:
def select_feat(train_data, valid_data, select_all=True):
    '''Selects useful features to perform regression'''
    sc = StandardScaler()
    train_data = sc.fit_transform(train_data)
    valid_data = sc.transform(valid_data)
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid = train_data[:,:-1], valid_data[:,:-1]
    

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = select_feature # TODO: Select suitable feature columns.
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], y_train, y_valid

In [459]:
class KGMTL_Data(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)
        

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)


    

In [460]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days) 
# test_data size: 1078 x 117 (without last day's positive rate)
# train_data, test_data = pd.read_csv('../LiterallyWikidata/files_needed/train_attri_data.csv',sep='\t').values, 
# pd.read_csv('../LiterallyWikidata/files_needed/test_attri_data.csv',sep='\t').values
train_data, valid_data = train_valid_split(x_list, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} """)
# test_data size: {test_data.shape}""")


# Select features
x_train, x_valid, y_train, y_valid = select_feat(train_data, valid_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset = KGMTL_Data(x_train, y_train), \
                                            KGMTL_Data(x_valid, y_valid)

print('train_dataset', train_dataset[0])

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)
# test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (142, 87) 
valid_data size: (15, 87) 
number of features: 21
train_dataset (tensor([-0.1708, -0.2967, -0.1158, -0.1861, -0.5509,  0.3183,  0.3378, -0.6285,
        -1.5861, -0.3149, -0.1587, -0.0197, -0.0379,  0.1674,  0.1237,  0.1008,
        -1.5408, -1.4524, -1.9680, -1.4826, -1.6331]), tensor(-0.2249))


In [461]:
train_dataset[5]

(tensor([-0.2469, -0.4112, -0.0890, -0.1975, -0.5994,  0.2029, -0.1439, -0.7458,
          0.5574,  0.5738,  0.3054, -0.6115, -0.5746, -0.9687, -0.9789, -0.5979,
          2.1452, -2.5163, -3.2250, -2.4723,  2.4697]),
 tensor(-0.2451))

In [462]:
class NeuralNet(nn.Module):
    ''' A simple fully-connected deep neural network '''
    def __init__(self, input_dim):
        super(NeuralNet, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(input_dim, 21),
            nn.ReLU(),
            nn.Dropout(0.5),
#             nn.Linear(20,10),
#             nn.ReLU(),
            nn.Linear(21, 1)
        )

        # Mean squared error loss
        self.criterion = nn.MSELoss(reduction='mean')

    def forward(self, x):
        ''' Given input of size (batch_size x input_dim), compute output of the network '''
        x = self.layers(x)
        x = x.squeeze(1)
        return x

    def cal_loss(self, pred, target):
        ''' Calculate loss '''
        # TODO: you may implement L1/L2 regularization here
        return self.criterion(pred, target)

# **Preprocess**

We have three kinds of datasets:
* `train`: for training
* `dev`: for validation
* `test`: for testing (w/o target value)

In [463]:
loss_record={'train': [], 'dev': [],'mean_train_loss':[],'mean_valid_loss':[]} 

def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9, weight_decay=1e-6) 

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    writer = SummaryWriter() # Writer of tensoboard
    if not os.path.isdir('./models_var'):
        os.mkdir('./models_var') # Create directory of saving models.
    
    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)   
            print(f'-------predict: {pred}, y: {y}----------') 
            #x_constraint = torch.tensor([ (y[i] - x[i][0]*x[i][18]) ** 2 for i in range(len(x))])
            x_constraint = torch.tensor([x[i][pop_idx]*x[i][gdp_per] for i in range(len(x))])
            print(x_constraint)
            x_constraint = x_constraint.to(device)          
            loss = criterion(pred, y) + criterion(pred, x_constraint)
            #loss = criterion(pred, y) 
            # criterion(pred,x_constraint)
                # ((pred-x[0]*x[18])**2) 

            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record["train"].append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record["train"])/len(loss_record["train"])
        writer.add_scalar('Loss/train', mean_train_loss, step)
        loss_record['mean_train_loss'].append(mean_train_loss)

        model.eval() # Set your model to evaluation mode.
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                # print(f'x: {x}')
                loss = criterion(pred, y)

            loss_record["dev"].append(loss.item())
            
        mean_valid_loss = sum(loss_record["dev"])/len(loss_record["dev"])
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)
        loss_record['mean_valid_loss'].append(mean_valid_loss)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

## **Validation**

In [464]:
def dev(dv_set, model, device):
    model.eval()                                # set model to evalutation mode
    total_loss = 0
    for x, y in dv_set:                         # iterate through the dataloader
        x, y = x.to(device), y.to(device)       # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)# forward pass (compute output)
            #print(pred)
            mse_loss = model.cal_loss(pred, y)  # compute loss
            print(mse_loss)
        total_loss += mse_loss.detach().cpu().item() * len(x)  # accumulate loss
    total_loss = total_loss / len(dv_set.dataset)              # compute averaged loss

    return total_loss

## **Testing**

In [465]:
device = get_device()                 # get the current available device ('cpu' or 'cuda')
#os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/

# TODO: How to tune these hyper-parameters to improve your model's performance?
config = {
    'seed': 80215,      # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features.
    'n_epochs': 50,                # maximum number of epochs
    'batch_size': 141,               # mini-batch size for dataloader
    'learning_rate':1e-3,
    'early_stop': 10,               # early stopping epochs (the number epochs since your model's last improvement)
    'save_path': './models_var/model_gdp_no_cons.pt' , # your model will be saved here
    'valid_ratio': 0.1,   # validation_size = train_size * valid_ratio
}


# **Load data and model**

In [466]:
model = NeuralNet(input_dim=x_train.shape[1]).to(device)  # Construct model and move to device
print(model)

NeuralNet(
  (layers): Sequential(
    (0): Linear(in_features=21, out_features=21, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=21, out_features=1, bias=True)
  )
  (criterion): MSELoss()
)


# **Start Training!**

In [467]:
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/50]: 100%|██████████████████████████████████████████████████████| 2/2 [00:00<00:00, 32.01it/s, loss=0.464]


-------predict: tensor([-0.2672,  0.1972,  0.2223,  0.2213,  0.4988, -0.2635,  0.1044, -0.0908,
         0.2254,  0.2885,  0.4322,  0.7681, -0.0205,  0.6254,  0.2754, -0.3920,
         0.1301,  0.6112,  0.0498, -0.3820,  0.1285,  0.4366,  0.0991,  0.3344,
         0.2212,  0.2757, -0.2075,  0.1163,  0.5275,  0.4677,  0.2288,  1.0191,
         0.3684, -0.1631,  0.0926,  0.3659, -0.0996,  0.4232,  0.1880,  0.3865,
         0.1612,  0.4054,  0.2070, -0.1229,  0.4490,  0.2506,  0.3447,  0.5494,
         0.2211,  0.2592,  0.1252,  0.2614,  0.0551,  0.5209,  0.0999,  0.1336,
         0.0840,  0.1445,  0.4245,  1.5251,  0.3617,  0.8709,  0.1748,  0.6865,
         0.2957,  0.0406, -0.4904, -0.1823,  0.1664,  0.4909,  0.1708,  0.2483,
         0.4104,  0.2096,  0.3806,  0.4941,  0.1236,  0.3923,  0.4880,  0.2490,
         0.5151,  0.3653, -0.0429, -0.2162, -0.1947,  0.7083, -0.1541,  0.2042,
        -0.2021,  0.2649,  0.3253,  0.6084,  2.1907,  0.3737,  1.0690,  0.4265,
         0.1090,  0.3641

  0%|                                                                                        | 0/2 [00:00<?, ?it/s]

-------predict: tensor([ 0.4726,  0.4325,  0.0763,  0.6669,  0.0965,  0.3843,  0.2821,  0.3977,
         0.2448,  0.0825,  0.2800,  0.1647,  0.2558,  0.3317,  0.2566, -0.0866,
         0.0931,  0.1724,  0.7773,  0.1392,  0.4585,  0.3394,  0.1685,  0.3403,
         2.5054,  0.3206,  0.2442,  0.0444, -0.0882, -0.1145,  0.1186,  0.3894,
        -0.1489,  0.0173,  0.0297,  0.1438,  0.1203,  0.2252,  0.1416,  0.4494,
         0.4780,  0.1004, -0.1861,  0.2277, -0.3130,  0.2130, -0.0344,  0.2966,
         0.4489,  0.5486,  0.2995,  0.1532,  0.0626,  0.3162,  0.2137,  0.0319,
         0.3111, -0.0535,  0.0313,  0.3362,  0.2111, -0.1591,  0.3250,  0.3203,
        -0.3703,  0.1770,  0.1222,  0.1205,  0.0715,  0.4472,  0.3242, -0.0166,
         0.0909,  0.2868,  0.1873,  0.3871,  0.2077, -0.3332,  0.1718, -0.1222,
        -0.3405,  0.2964,  1.0411,  0.8570,  0.7908,  0.6713,  0.0178,  0.4656,
         0.3058,  0.1810,  0.0225,  0.1197,  0.5689,  0.4451,  0.4565,  0.0509,
        -0.0253, -0.3904

Epoch [2/50]: 100%|█████████████████████████████████████████████████████| 2/2 [00:00<00:00, 46.49it/s, loss=0.0253]


...

-------predict: tensor([0.0514], device='cuda:0', grad_fn=<SqueezeBackward1>), y: tensor([-0.2446], device='cuda:0')----------
tensor([0.2717])
Epoch [49/50]: Train loss: 1.0982, Valid loss: 0.0669
Saving model with loss 0.067...


  0%|                                                                                        | 0/2 [00:00<?, ?it/s]

-------predict: tensor([-8.0634e-02, -1.6896e-01, -2.9192e-01,  1.2267e-01,  4.3612e-02,
         8.9554e-02, -1.0587e-01, -2.9388e-01,  5.4924e-01, -1.7043e-01,
         2.0933e-02,  1.1682e-01,  1.1625e-01,  7.3957e-02, -3.8395e-01,
        -1.0796e-01,  2.4955e-01, -1.8434e-02,  1.0674e-01, -1.1780e-01,
        -3.1818e-01, -1.3623e-01,  3.1929e-02, -1.2703e-01, -1.1217e-01,
         4.5928e-01,  2.8414e-02,  4.9261e-02, -2.0218e-01, -1.5846e-01,
        -2.0593e-01,  2.7765e-01,  2.1872e-02, -1.9232e-01, -1.3625e-01,
        -7.6961e-02,  1.0137e-01,  2.5288e-01, -2.2454e-01, -1.0670e-01,
         3.4179e-01, -4.5766e-02, -3.3145e-02, -3.5005e-02, -5.4889e-01,
        -6.0037e-02, -3.2841e-01, -5.5517e-02,  1.3283e-01, -1.2957e-01,
         2.1392e-01, -5.5754e-02,  4.5074e-02, -9.0925e-02, -2.5060e-01,
         6.4535e-01, -3.4792e-02, -1.2172e-01, -1.8165e-01, -2.3177e-01,
         2.4698e-01,  9.4691e-02, -6.4938e-02, -2.8586e-01,  2.2700e-01,
        -8.6346e-02,  4.0319e-02, -

Epoch [50/50]: 100%|█████████████████████████████████████████████████████| 2/2 [00:00<00:00, 68.64it/s, loss=0.078]


tensor([ 7.2377e-02, -3.0378e-04, -7.0731e-02, -1.5848e-02,  3.2908e-01,
         5.5600e-01,  6.2135e-01, -2.8738e-02,  8.9280e-03,  3.8809e-03,
        -1.2856e-02, -4.9693e-01,  1.5892e-01, -2.1008e-02, -3.7727e-02,
        -7.1206e-02, -1.8274e-02,  1.4809e-01,  2.7265e-01,  2.4615e-01,
        -1.2122e-02,  6.0025e-02, -2.1879e-02, -2.5499e-01, -5.2776e-02,
         1.2040e-02,  2.6465e-01, -2.3377e-01,  5.1115e-02, -1.3775e-01,
        -3.0333e-02, -1.6254e-02,  2.8527e-01,  5.6118e-02,  4.2126e-02,
         2.7804e-01,  2.2071e+00,  2.2896e-01,  5.5103e-03,  3.6443e-01,
        -1.3127e-02,  1.1929e-02, -9.2765e-02, -4.1669e-02,  5.6202e-02,
         2.2287e-02,  4.5290e-02,  2.0601e-02,  2.2802e-01,  2.8127e-02,
         1.9887e-02, -1.2142e-01,  1.3043e+00, -5.7422e-02, -3.4175e-01,
        -7.2477e-02, -5.8953e-01,  2.9132e-01,  1.1882e-02,  2.8987e-02,
        -9.4473e-02, -4.7574e-02, -5.6639e-01, -2.4948e-01, -1.4991e-01,
        -2.2303e-01,  3.0004e-01,  1.9810e-01,  1.9

In [1]:
plot_learning_curve(loss_record, title='deep model')

NameError: name 'plot_learning_curve' is not defined

In [426]:
del model
model = NeuralNet(input_dim=x_train.shape[1]).to(device)
ckpt = torch.load(config['save_path'], map_location='cpu')  # Load your best model
model.load_state_dict(ckpt)
#plot_pred(dv_set, model, device)  # Show prediction on the validation set

<All keys matched successfully>

In [430]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
def eval_matrics(y_test, y_pred):

    MSE = mean_squared_error(y_test, y_pred)
    print('MSE=',MSE)
    RMSE =np.sqrt(MSE)
    print('RMSE=',RMSE)
    MAE= mean_absolute_error(y_test, y_pred)
    print('MAE=',MAE)

    R2=1-MSE/np.var(y_test)
    print("R2=", R2)

In [431]:

def test(tt_set, model, device):
    model.eval()                                # set model to evalutation mode
    preds = []; y_b=[]
    for x,y in tt_set:                            # iterate through the dataloader
        x ,y = x.to(device), y.to(device)                          # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            preds.append(pred.detach().cpu())
            y_b.append(y.detach().cpu())   # collect prediction
    preds = torch.cat(preds, dim=0).numpy().reshape(-1,1)     # concatenate all predictions and convert to a numpy array
    y_b= torch.cat(y_b,0).numpy().reshape(-1,1) 
    table  = np.concatenate((preds, y_b),axis=1)
    eval_matrics(y_b,preds)
    return table

In [469]:
preds = test(valid_loader, model, device) 

MSE= 0.028597383
RMSE= 0.1691076
MAE= 0.13527282
R2= -0.3172532320022583


# **Testing**
The predictions of your model on testing set will be stored at `pred.csv`.

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    print('Saving results to {}'.format(file))
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

preds = test(valid_loader, model, device)  # predict COVID-19 cases with your model
print('pred')         # save prediction file to pred.csv

# **Hints**

## **Simple Baseline**
* Run sample code

## **Medium Baseline**
* Feature selection: 40 states + 2 `tested_positive` (`TODO` in dataset)

## **Strong Baseline**
* Feature selection (what other features are useful?)
* DNN architecture (layers? dimension? activation function?)
* Training (mini-batch? optimizer? learning rate?)
* L2 regularization
* There are some mistakes in the sample code, can you find them?

# **Reference**
This code is completely written by Heng-Jui Chang @ NTUEE.  
Copying or reusing this code is required to specify the original author. 

E.g.  
Source: Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)


In [94]:
## Load pretrain embedding
emb_ent = torch.load('../LiterallyWikidata/files_needed/pretrained_kge/pretrained_complex_entemb.pt')
list_ent_ids =[]
with open('../LiterallyWikidata/files_needed/list_ent_ids.txt','r') as f:
    for line in f:
        list_ent_ids.append(line.strip())
## Preparing ent embedding
ent2idx = {e:i for i,e in enumerate(list_ent_ids)}
attri_data['ent_idx']= attri_data['e'].map(ent2idx)
embedding_e = torch.nn.Embedding.from_pretrained(emb_ent)
input_e = torch.LongTensor(attri_data['ent_idx'].to_numpy())

entity_embedding = embedding_e(input_e)
## Preparing att embedding
# att2idx = {a:i for i,a in enumerate(attri_data['a'].unique())}
attri_data['a_idx']=attri_data['a'].map(att2idx)
embedding_a = torch.nn.Embedding(len(att2idx),128,padding_idx=0)
input_a = torch.LongTensor(attri_data['a'].to_numpy())

attribute_embedding = embedding_a(input_a)
## concat two embedding
x_data = torch.cat([entity_embedding,attribute_embedding],dim=1).detach().numpy()

